In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install transformers

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
from transformers import (EncoderDecoderModel,
                          PreTrainedModel,
                          BertTokenizer,
                          BertGenerationEncoder,
                          BertGenerationDecoder)

In [4]:
import os
import torch
from torch import nn
from tqdm import tqdm

### Determining device to run on

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


### Reading hyperparameters from SageMaker env variables

In [6]:
model_type = os.environ.get('SM_HP_MODEL_TYPE', 'bert-base-uncased')
data_loc = os.environ.get('SM_HP_DATA_LOC', '../data')
epochs = int(os.environ.get('SM_HP_EPOCHS', 2))
batch = int(os.environ.get('SM_HP_BATCH', 32))
lr = float(os.environ.get('SM_HP_LR', 1e-5))
train_remotely = bool(int(os.environ.get('SM_HP_TRAIN_REMOTELY', 1)))
is_sagemaker_estimator = 'TRAINING_JOB_NAME' in os.environ  # This code is running on the remote SageMaker estimator machine

In [7]:
BOS_TOKEN_ID = 101
EOS_TOKEN_ID = 102

### Initializing data loaders for Oxford2019 dataset

In [8]:
from dataset import Oxford2019Dataset
from torch.utils.data import DataLoader

def make_data_loader(filename: str, file_loc: str = os.path.join(data_loc, 'Oxford-2019')) -> DataLoader:
    dataset = Oxford2019Dataset(data_loc=os.path.join(file_loc, filename))
    data_loader = DataLoader(dataset, batch_size=batch, shuffle=True)
    return data_loader

train_set = make_data_loader('train.txt')
test_set = make_data_loader('test.txt')
valid_set = make_data_loader('valid.txt')


### Function to run through one epoch
This function is used in training, validation, and testing phases.

In [10]:
from typing import Callable

def run(model: nn.Module,
        data_loader: DataLoader,
        tokenizer: BertTokenizer,
        post_hook: Callable = lambda b: ''):

    loss = 0
    for words, examples, defs, _ in tqdm(data_loader, disable=is_sagemaker_estimator):
        input_ids = tokenizer(examples,
                              add_special_tokens=False,
                              padding=True,
                              truncation=True,
                              return_tensors="pt").input_ids
        output_ids = tokenizer(defs,
                               padding=True,
                               truncation=True,
                               return_tensors="pt").input_ids
        
        input_ids = input_ids.to(device)
        output_ids = output_ids.to(device)


        outputs = model(input_ids=input_ids,
                        decoder_input_ids=output_ids,
                        labels=output_ids,
                        return_dict=True)
        batch_loss = outputs.loss
        loss += batch_loss.item()

        post_hook(batch_loss)
    return loss

### Training loop function

In [11]:
from transformers import AdamW


def train(epochs: int, train_data_loader: DataLoader, valid_data_loader: DataLoader = None, model: nn.Module = None):
    if model is None:
        encoder = BertGenerationEncoder.from_pretrained(model_type,
                                                        bos_token_id=BOS_TOKEN_ID,
                                                        eos_token_id=EOS_TOKEN_ID) # add cross attention layers and use BERT’s cls token as BOS token and sep token as EOS token

        decoder = BertGenerationDecoder.from_pretrained(model_type,
                                                        add_cross_attention=True,
                                                        is_decoder=True,
                                                        bos_token_id=BOS_TOKEN_ID,
                                                        eos_token_id=EOS_TOKEN_ID)
        model = EncoderDecoderModel(encoder=encoder, decoder=decoder).to(device)


    optimizer = AdamW(model.parameters(), lr=lr)

    tokenizer = BertTokenizer.from_pretrained(model_type)
    

    def update_weights(batch_loss):
        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    for i in range(epochs):
        model.train()
        train_loss = run(model, train_data_loader, tokenizer, update_weights)

        if valid_data_loader is not None:
            with torch.no_grad():
                model.eval()
                val_loss = run(model, valid_data_loader, tokenizer)
        else:
            val_loss = 'N/A'
        
        msg = f'train_error={train_loss};  valid_error={val_loss};'
        print(msg)
        !echo '{msg}' >> log.txt
    return model

### Quick sanity check for the training loop

In [12]:
if not is_sagemaker_estimator:
    encoder = BertGenerationEncoder.from_pretrained(model_type,
                                                    bos_token_id=BOS_TOKEN_ID,
                                                    eos_token_id=EOS_TOKEN_ID) # add cross attention layers and use BERT’s cls token as BOS token and sep token as EOS token

    decoder = BertGenerationDecoder.from_pretrained(model_type,
                                                    add_cross_attention=True,
                                                    is_decoder=True,
                                                    bos_token_id=BOS_TOKEN_ID,
                                                    eos_token_id=EOS_TOKEN_ID)
    model = EncoderDecoderModel(encoder=encoder, decoder=decoder).to(device)    
    
    train_file = os.path.join(data_loc, 'Oxford-2019', 'train.txt')
    tiny_size = batch * 5
    tiny_file = os.path.join(data_loc, 'Oxford-2019', 'tiny.txt')
    !head -n {tiny_size} {train_file} > {tiny_file}
    tiny_set = make_data_loader('tiny.txt')
    model = train(epochs=2, train_data_loader=tiny_set, valid_data_loader=tiny_set, model=model)

100%|██████████| 5/5 [00:01<00:00,  2.84it/s]

Epoch 0 | train loss 49.14696216583252, val loss 46.320258140563965



100%|██████████| 5/5 [00:01<00:00,  2.74it/s]

Epoch 1 | train loss 44.41345024108887, val loss 40.74758243560791


### Function for saving the model

In [13]:
def save_model(model: PreTrainedModel):
    out_loc = '/opt/ml/model' if is_sagemaker_estimator else '.'
    !mkdir -p {out_loc}

    model.save_pretrained(out_loc)

    !cp main_bert.py {out_loc}
    !cp main_bert.ipynb {out_loc}
    !cp log.txt {out_loc}

### Training
Training can be done either on the same machine where notebook is running or remotely on SageMaker estimator

In [15]:
import sagemaker
from sagemaker.pytorch import PyTorch

if is_sagemaker_estimator:
    model = train(epochs=epochs, train_data_loader=train_set, valid_data_loader=valid_set)
    save_model(model)
elif train_remotely:
    role = sagemaker.get_execution_role()
    output_path = f's3://chegg-ds-data/oboiko/wdm-output'

    pytorch_estimator = PyTorch(entry_point='train.sh',
                                base_job_name='wdm-1',
                                role=role,
                                train_instance_count=1,
                                train_instance_type='ml.g4dn.2xlarge',  # GPU instance
                                train_volume_size=50,
                                train_max_run=86400,  # 24 hours
                                hyperparameters={
                                  'model_type': 'bert-base-uncased',
                                  'data_loc': '/opt/data',
                                  'batch': 50,
                                  'epochs': 1,
                                  'lr': 1e-5,
                                  'train_remotely': 0
                                },
                                framework_version='1.6.0',
                                py_version='py3',
                                source_dir='.',  # This entire folder will be transferred to training instance
                                debugger_hook_config=False,
                                output_path=output_path,  # Model files will be uploaded here
                                image_name='954558792927.dkr.ecr.us-west-2.amazonaws.com/sagemaker/wdm:latest',
                                metric_definitions=[
                                    {'Name': 'train:error', 'Regex': 'train_error=(.*?);'},
                                    {'Name': 'validation:error', 'Regex': 'valid_error=(.*?);'}
                                ]
                     )

    pytorch_estimator.fit('s3://chegg-ds-data/oboiko/wdm/dummy.txt', wait=True)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-31 03:44:25 Starting - Starting the training job...
2020-10-31 03:44:27 Starting - Launching requested ML instances......
2020-10-31 03:45:36 Starting - Preparing the instances for training......
2020-10-31 03:46:32 Downloading - Downloading input data
2020-10-31 03:46:32 Training - Downloading the training image.........
2020-10-31 03:48:19 Training - Training image download completed. Training in progress..2020-10-31 03:48:21,799 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": null,
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "lr": 1e-05,
        "batch": 50,
        "model_type": "bert-base-uncased",
        "data_loc": "/opt/data",
        "epochs": 1,
        "train_remotely": 0
    },
    "input_config_dir": "/opt/ml/input/config",
    "in

TODO: For loss function... instead of doing log_softmax, do MSE with actual GloVe vector and minimize this loss function.
Then for BLEU evaluation, you'll need a function to find the closest vector to the one produced by the model.

Interesting to compare these results to log_softmax